Preparing Nott Promethion datasets for pipeline

In [11]:
cd /lscratch/ndolson/GM24385_Prom_Notts_1/2-A3-D3/reads

/lscratch/ndolson/GM24385_Prom_Notts_1/2-A3-D3/reads


In [15]:
ls

0/   15/  21/  28/  34/  40/  47/  53/  6/   66/  72/  79/  85/  91/  98/
1/   16/  22/  29/  35/  41/  48/  54/  60/  67/  73/  8/   86/  92/  99/
10/  17/  23/  3/   36/  42/  49/  55/  61/  68/  74/  80/  87/  93/
11/  18/  24/  30/  37/  43/  5/   56/  62/  69/  75/  81/  88/  94/
12/  19/  25/  31/  38/  44/  50/  57/  63/  7/   76/  82/  89/  95/
13/  2/   26/  32/  39/  45/  51/  58/  64/  70/  77/  83/  9/   96/
14/  20/  27/  33/  4/   46/  52/  59/  65/  71/  78/  84/  90/  97/


In [31]:
%%bash 
work_dir=/scratch/groups/msalit/nanopore/raw/GM24385_Prom_Notts_1
run_name=GM24385_Prom_Notts_1
run_dir=/scratch/groups/msalit/nanopore/raw/GM24385_Prom_Notts_1

## Looping through subread directories e.g. 0,1,2,...
for dir in {91..99};
do
    cd /lscratch/ndolson/GM24385_Prom_Notts_1/2-A3-D3/reads/

    echo "Read Dir:" ${dir}

    ## Grouping fast5s into 1000 read tar balls
    ls ${dir} > fast5_${dir}.txt
    lst_dir=$(pwd)
        split -l 1000 -d --additional-suffix=_fast5.lst fast5_${dir}.txt ${run_name}_${dir}_
    rm fast5_${dir}.txt        
    echo ${lst_dir}
    ls -lh

    for lst in *_fast5.lst;
        do
            echo "2-A3-D3-ext1" ${lst}
            out_tar=${lst%_fast5.lst}.tar
            cd ${lst_dir}/${dir}
        pwd
            tar -c --ignore-failed-read --file=${work_dir}/fast5/${out_tar} -T ${lst_dir}/${lst}
        rm ${lst_dir}/${lst}
    done
    echo "Done first loop"
    pwd
done


Read Dir: 91
/lscratch/ndolson/GM24385_Prom_Notts_1/2-A3-D3/reads
total 112M
drwxr-xr-x 2 ndolson msalit 1008K Jun 19  2018 0
drwxr-xr-x 2 ndolson msalit  1.1M Jun 19  2018 1
drwxr-xr-x 2 ndolson msalit  1.1M Jun 19  2018 10
drwxr-xr-x 2 ndolson msalit  1.1M Jun 19  2018 11
drwxr-xr-x 2 ndolson msalit  1.1M Jun 19  2018 12
drwxr-xr-x 2 ndolson msalit  1.1M Jun 19  2018 13
drwxr-xr-x 2 ndolson msalit  1.1M Jun 19  2018 14
drwxr-xr-x 2 ndolson msalit  1.1M Jun 19  2018 15
drwxr-xr-x 2 ndolson msalit  1.1M Jun 19  2018 16
drwxr-xr-x 2 ndolson msalit  1.1M Jun 19  2018 17
drwxr-xr-x 2 ndolson msalit  1.1M Jun 19  2018 18
drwxr-xr-x 2 ndolson msalit  1.1M Jun 19  2018 19
drwxr-xr-x 2 ndolson msalit  1.1M Jun 19  2018 2
drwxr-xr-x 2 ndolson msalit  1.1M Jun 19  2018 20
drwxr-xr-x 2 ndolson msalit  1.1M Jun 19  2018 21
drwxr-xr-x 2 ndolson msalit  1.1M Jun 19  2018 22
drwxr-xr-x 2 ndolson msalit  1.1M Jun 19  2018 23
drwxr-xr-x 2 ndolson msalit  1.1M Jun 19  2018 24
drwxr-xr-x 2 ndolson msali

Extract flowcell info

In [33]:
import h5py
import os
import tarfile

In [34]:
def extract_fast5_file(run_fast5_path):
    fast5_tar_files = os.listdir(run_fast5_path)
    fast5_tars = tarfile.open(run_fast5_path + "/" + fast5_tar_files[1])
    fast5_names = fast5_tars.getnames()
    fast5_tars.extract(fast5_names[1])
    return(fast5_names[1])

def get_fast5_info(fast5_file):
    ## Reading fast5 file
    f = h5py.File(fast5_file, 'r')
    
    ## Getting metadata
    ct_attrs = f["UniqueGlobalKey"]["context_tags"].attrs
    sk = ct_attrs["sequencing_kit"]
    ft = ct_attrs["flowcell_type"]
    ti_attrs = f["UniqueGlobalKey"]["tracking_id"].attrs
    dt = ti_attrs["device_type"]
    fc = ti_attrs["flow_cell_id"]
    
    print(f"{dt}\t{fc}\t{sk}\t{ft}")
    
    ## Removing fast5 file
    os.remove(fast5_file)

In [38]:
cd /scratch/groups/msalit/nanopore/raw

/scratch/groups/msalit/nanopore/raw


In [39]:
run_name = "GM24385_Prom_Notts_1"
run_fast5_path = f"{run_name}/fast5"
fast5_file = extract_fast5_file(run_fast5_path)
get_fast5_info(fast5_file)

b'promethion'	b'PAC17886'	b'psk-lsk109'	b'flo-pro001'
